In [1]:
# Get Training Data
#%pip show nlpaf
%pip install  -e ../

Obtaining file:///home/elba_ro/repos/github/conf22-style-transfer
  Attempting uninstall: iesta
    Found existing installation: iesta 1.3.18
    Uninstalling iesta-1.3.18:
      Successfully uninstalled iesta-1.3.18
  Running setup.py develop for iesta
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip show iesta


Name: iesta
Version: 1.3.16
Summary: Python package for Conf 2023 for INEFF-IFFE Style Transfer
Home-page: UNKNOWN
Author: Roxanne El Baff
Author-email: roxanne.elbaff@dlr.de
License: MIT
Location: /home/elba_ro/repos/github/conf22-style-transfer
Requires: empath, markupsafe, nlpaf, seaborn, spacy, tomli, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
from  iesta.machine_learning.dataloader import load_training_features_df
import iesta.properties

/home/elba_ro/repos/github/conf22-style-transfer/iesta_venv/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [3]:
from datasets.dataset_dict import DatasetDict


In [4]:
from datasets import load_dataset


In [5]:
from iesta.machine_learning.dataloader import IESTAData, LABELS
from iesta.machine_learning.huggingface_loader import IESTAHuggingFace


### Data Count for main

training      27522

validation     7962

test           3906


### Data Count for style classifier


training      11913

validation     3400

test           1691

In [6]:
conservative_data_obj = IESTAData(ideology="conservative", keep_labels = LABELS.EFF_INEFF)
conservative_hf = IESTAHuggingFace(conservative_data_obj)
conservative_w_effect = conservative_hf.upload_w_labels(is_for_style_classifier=False)
conservative_w_effect_for_sc = conservative_hf.upload_w_labels(is_for_style_classifier=True)


dataset was not found on hugging face.
creating huggingface dataset from local dataset
/home/elba_ro/repos/github/conf22-style-transfer/iesta/../data/splitted_conservative_debate_arguments_effect_test0.3_random2.parquet


Pushing split test to the Hub.


The file for data_w_splits_df already exists
Before filtering effects 101932
After filtering effects 84711
The columns are ['id', 'debate_id', 'p_name', 'effects', 'effect_count', 'top_effect', 'label', 'category', 'round', 'argument', 'split', 'is_for_eval_classifier', 'text']


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split training to the Hub.


...

Generating test split:   0%|          | 0/2474 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5082 [00:00<?, ? examples/s]

Generating training split:   0%|          | 0/17743 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /home/elba_ro/.cache/huggingface/datasets/notaphoenix___parquet/notaphoenix--debateorg_w_effect_for_conservative_subset-f9040a5a0d89b057/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
liberal_data_obj = IESTAData(ideology="liberal", keep_labels = LABELS.EFF_INEFF)
liberal_hf = IESTAHuggingFace(liberal_data_obj)
liberal_w_effect = liberal_hf.upload_w_labels(is_for_style_classifier=False)
liberal_w_effect_for_sc = liberal_hf.upload_w_labels(is_for_style_classifier=True)


dataset was not found on hugging face.
creating huggingface dataset from local dataset
/home/elba_ro/repos/github/conf22-style-transfer/iesta/../data/splitted_liberal_debate_arguments_effect_test0.3_random2.parquet


Pushing split test to the Hub.


The file for data_w_splits_df already exists
Before filtering effects 66844
After filtering effects 56394
The columns are ['id', 'debate_id', 'p_name', 'effects', 'effect_count', 'top_effect', 'label', 'category', 'round', 'argument', 'split', 'is_for_eval_classifier', 'text']


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split training to the Hub.


...

Generating test split:   0%|          | 0/1717 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3416 [00:00<?, ? examples/s]

Generating training split:   0%|          | 0/11719 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /home/elba_ro/.cache/huggingface/datasets/notaphoenix___parquet/notaphoenix--debateorg_w_effect_for_liberal_subset-dee676fc10300bce/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:

def get_training(ideology, evaluation_classfier_data_flag= IESTAData._WITHOUT_EVAL_CLASSIFIER, discard_okay: bool = True):
    _, feature_df = load_training_features_df(ideology)

    training = feature_df
    print(len(training))

    if evaluation_classfier_data_flag == IESTAData._ONLY_EVAL_CLASSIFIER_:
        training = training[training['is_for_eval_classifier'] == True]
        print(len(training))
    elif evaluation_classfier_data_flag == IESTAData._WITHOUT_EVAL_CLASSIFIER:
        training = training[training['is_for_eval_classifier'] == False]
        print(len(training))
    if discard_okay:
        training = training[training["effect"]!="okay"]
    print(f"Included effects are: {training.effect.unique()}")
    return training

liberal_training = get_training( "liberal")
liberal_training.columns.tolist

In [ ]:
liberal_iesta = IESTAData(ideology="liberal", keep_labels = LABELS.EFF_INEFF)
data_w_splits_df, _ = liberal_iesta.split_iesta_dataset_by_debate()


In [ ]:
default_data = data_w_splits_df[data_w_splits_df['is_for_eval_classifier'] == False]
style_classifier_data = data_w_splits_df[data_w_splits_df['is_for_eval_classifier'] == True]

In [ ]:
import pandas as pd

print(pd.crosstab(default_data['split'], default_data['effect']))
print(pd.crosstab(style_classifier_data['split'], style_classifier_data['effect']))
print()
print(default_data['split'].value_counts())
print(style_classifier_data['split'].value_counts())

In [ ]:
from sklearn.model_selection import train_test_split


unique_debates = liberal_training.debate_id.unique()

training_debates, testval_debates = train_test_split(unique_debates, test_size=0.3,
                 random_state=42,
                 shuffle=True)

def _add_splits(row, training_debates, testval_debates):
    if row['debate_id'] in training_debates:
        row["classifier_split"] = "train"
    elif row['debate_id'] in testval_debates:
        row["classifier_split"] = "test"
    else:
        assert False
    return row 

liberal_training = liberal_training.apply(_add_splits, args=(training_debates,testval_debates, ),axis=1)

In [ ]:
liberal_training.classifier_split.value_counts()
import pandas as pd

pd.crosstab(liberal_training.classifier_split, liberal_training.binary_effect)

In [ ]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
def filter_cols(cols):
    prefices = ["mpqa", "hedge", "emotion", "toxicity", "mpqa", "empath"]
    keep = [c for c in cols if c.split("_")[0] in prefices and not c.endswith("_count")]
    return keep

def filter_feature_df(df: pd.DataFrame, class_col: str = "effect", split_col: str = "classifier_split") : # binary_effect
    feature_cols = filter_cols(df.columns.tolist())
    df_ = df[feature_cols+[class_col, split_col]].copy()
    numeric_cols = df_.select_dtypes(include=np.number).columns.tolist()
    df_ = df_[numeric_cols+[class_col, split_col]]
    
    df_.fillna(0.0, inplace=True)

    # transform y to numeric
    le = LabelEncoder()
    le.fit(df_[class_col])
    df_[class_col] = le.transform(df_[class_col])
    #X = df_.values
    #y_str = df[class_].values


    #def to_numeric(x):
    #    return 0 if x == "ineffective" else (1 if x == "effective" else 2)
    #y = [to_numeric(x) for x in y_str]
    
    return df_, numeric_cols

In [ ]:
le = LabelEncoder()
le.fit(["paris", "paris", "tokyo", "amsterdam"])
list(le.classes_)
le.transform(["amsterdam", ])


In [ ]:


_df, numeric_cols = filter_feature_df(liberal_training, class_col = "binary_effect", split_col = "classifier_split")

In [ ]:
_df.binary_effect.value_counts()

In [ ]:
from nlpaf.ml.feature_based import FeatureBasedML

trainer = FeatureBasedML(y_col="binary_effect", dataset=_df, split_label_name="classifier_split", training_cols=numeric_cols, remove_outliers=False, normalize=False)

In [ ]:
dummy = trainer.train_baseline()

In [ ]:
xfold_results = trainer.train_xfold()

In [ ]:
xfold_results.keys()

In [ ]:
xfold_results

In [ ]:
trainer_nooutliers = FeatureBasedML(y_col="binary_effect", dataset=_df, split_label_name="classifier_split", training_cols=numeric_cols, remove_outliers=True, normalize=True)

In [ ]:
xfold_nooutlier_results = trainer_nooutliers.train_xfold()

In [ ]:
from nlpaf.ml.feature_based import FeatureBasedML

trainer_ensemble = FeatureBasedML(y_col="binary_effect", dataset=_df, split_label_name="classifier_split", training_cols=numeric_cols, remove_outliers=True, normalize=True)

In [ ]:
from nlpaf.ml import traditional_trainer as tt

results = tt.train_ensemble(trainer_ensemble.X_train, trainer_ensemble.X_test, trainer_ensemble.y_train, trainer_ensemble.y_test)


"""
          precision    recall  f1-score   support

           0       0.06      0.12      0.08       563
           1       0.95      0.91      0.93     11554

    accuracy                           0.87     12117
   macro avg       0.51      0.51      0.50     12117
weighted avg       0.91      0.87      0.89     12117


time: 2222m 7.3s
"""
